# Demonstrate Rejection Sampling
This performs rejection sampling on the double well

In [ ]:
using Random
using Plots
using QuadGK
using Distributions

In [ ]:
# for readability, these are good settings to use
default(xtickfontsize=14,  ytickfontsize=14, 
    guidefontsize=14, legendfontsize=12, lw=2,ms=8)

# Example
Start with the __unnormalized__ PDF,
$$
f(x) \propto \exp(-(x^2 -1)^2)1_{[-2,2]}(x)
$$
To use inverse function sampling on this problem, we will first need to, numerically, compute the normalization, $Z$,
such that
$$
\int Z^{-1}\exp(-(x^2 -1)^2)1_{[-2,2]}(x)dx = 1.
$$
So
$$
Z = \int_{-2}^2 \exp(-(x^2 -1)^2) dx.
$$

In [ ]:
Z = quadgk(x->exp(-(x^2-1)^2), -2,2)[1];
@show Z;

Note that extract the first return with the `[1]`, as teh second value is an estimate of the error, which we do not need right now.

In [ ]:
# define the density
f = x->exp(-(x^2-1)^2)/Z;

In [ ]:
xx = LinRange(-2,2,200);
C = 2.1;

plot(xx, f.(xx), label="Target Density")
plot!(xx, (1/4) *  ones(size(xx)),label="Proposal")
plot!(xx, C* (1/4) *  ones(size(xx)),label="C × Proposal")
xlabel!("x")
ylabel!("Probability")

In [ ]:
unif_dist = Uniform(-2,2);

function double_well_rejection_sample(;max_iter = 1000)
    for _ in 1:max_iter
        X = rand(unif_dist);
        Y = rand(Uniform(0, C))
        if(Y <= f(X))
            return X
        end
    end
    println("ERROR: Failed to sample")
    return -10;
end

In [ ]:
n = 10^4;

Random.seed!(100);
samples = [double_well_rejection_sample() for _ in 1:n];

In [ ]:
xx = LinRange(-2,2,200);
histogram(samples, label="Samples", norm =:pdf)
plot!(xx, f.(xx), label="Density")
xlabel!("x")
ylabel!("Probability")